# Convert a stream's flowline into broken reaches

In [1]:
# separate environment required for qgis processing
import processing
from qgis.core import QgsApplication
from processing.core.Processing import Processing
from qgis.analysis import QgsNativeAlgorithms
from qgis.core import QgsProperty

from pathlib import Path


qgs = QgsApplication([], False)
qgs.initQgis()
Processing.initialize()
QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms())

Application path not initialized
qt.qpa.fonts: Populating font family aliases took 96 ms. Replace uses of missing font family "Open Sans" with one that exists to avoid this cost. 
Logged warning: Duplicate provider native registered


False

In [2]:
shapefile_io_dir = Path('/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/01-Hydrothermal History/Data/GIS/shapefiles/flowlines_to_reaches')
# shapefile_io_dir = Path('/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/01-Hydrothermal History/Data/GIS/shapefiles/flowlines_to_reaches')
# reservoirs_shapefile = Path('/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/01-Hydrothermal History/Data/GIS/raw/gee_basin_params/basin_reservoirs.shp')
# reservoirs_shapefile = Path('/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/01-Hydrothermal History/Data/GIS/raw/gee_basin_params/basin_reservoirs.shp')
reservoirs_shapefile = Path('/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/01-Hydrothermal History/Data/GIS/shapefiles/CRBReservoirs.shp')

input_flowline = shapefile_io_dir / 'columbia_tributaries.shp'
interpolatedPoints = shapefile_io_dir / 'interpolatedPoints.shp'
transects = shapefile_io_dir / 'transects.shp'
reachLines = shapefile_io_dir / 'reachLines.shp'
filteredReachLines = shapefile_io_dir / 'filteredReachLines.shp'
bufferedReaches = shapefile_io_dir / 'bufferedReaches.shp'
finalFilteredReaches = shapefile_io_dir / 'finalFilteredReaches.shp'

temporaryFile = shapefile_io_dir / 'temp.shp'

# bufferDistance = 3000 # meters (half on each side of the line)

# koppen_raster = Path("/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/01-Hydrothermal History/Data/GIS/raw/Beck_KG_V1/Beck_KG_V1_present_0p083.tif")
narivs = Path("/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/01-Hydrothermal History/Data/GIS/raw/namerica/narivs.shp")
grwl = Path("/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/01-Hydrothermal History/Data/GIS/raw/GRWL_summaryStats_V01.01/GRWL_summaryStats.shp")

In [3]:
def interpolatePoints(input, output, distance=10000, startOffset=0, endOffset=0):
    """
    Interpolate points along a line at a specified distance
    Reference:
        https://docs.qgis.org/3.28/en/docs/user_manual/processing_algs/qgis/vectorgeometry.html#points-along-geometry
        https://gis.stackexchange.com/questions/380361/creating-perpendicular-lines-on-line-using-qgis

    Parameters
    ----------
    input : str or Path object
        Path to the input shapefile
    output : str or Path object
        Path to the output shapefile
    distance : int
        Distance between points
    startOffset : int or float
        Offset from the start of the line
    endOffset : int or float
        Offset from the end of the line

    Returns
    -------
    None
    """
    processing.run(
        "native:pointsalonglines",
        {
            "INPUT": str(input),
            "DISTANCE": distance,
            "START_OFFSET": startOffset,
            "END_OFFSET": endOffset,
            "OUTPUT": str(output),
        },
    )


def createTransects(input, output, transectLength=40):
    """
    Create transects perpendicular to a line
    Reference:
        https://gis.stackexchange.com/questions/380361/creating-perpendicular-lines-on-line-using-qgis

    Parameters
    ----------
    input : str or Path object
        Path to the input shapefile (points along a line)
    output : str or Path object
        Path to the output shapefile
    transectLength : int or float
        Length of the transect

    Returns
    -------
    None
    """
    processing.run(
        "native:geometrybyexpression",
        {
            "INPUT": str(input),
            "OUTPUT_GEOMETRY": 1,
            "WITH_Z": False,
            "WITH_M": False,
            "EXPRESSION": f'extend(\n   make_line(\n      $geometry,\n       project (\n          $geometry, \n          {transectLength}, \n          radians("angle"-90))\n        ),\n   {transectLength},\n   0\n)',
            "OUTPUT": str(output),
        },
    )


def createReachLines(input, splitLine, output):
    """
    Create reach lines from a line and a split line
    Reference:
        https://docs.qgis.org/3.28/en/docs/user_manual/processing_algs/qgis/vectoroverlay.html#split-with-lines
        https://gis.stackexchange.com/questions/380361/creating-perpendicular-lines-on-line-using-qgis

    Parameters
    ----------
    input : str or Path object
        Path to the input shapefile (river flowline)
    splitLine : str or Path object
        Path to the split lines shapefile (transects)
    output : str or Path object
        Path to the output shapefile (reach lines)

    Returns
    -------
    None
    """
    processing.run(
        "native:splitwithlines",
        {
            "INPUT": str(input),
            "LINES": str(splitLine),
            "OUTPUT": str(output),
        },
    )


def joinAttributesByNearest(
    input1,
    input2,
    output,
    # fieldsToCopy=[
    #     "DISCHARGE",
    #     "WIDTH",
    #     "WIDTH5",
    #     "WIDTH95",
    #     "DEPTH",
    #     "DEPTH5",
    #     "DEPTH95",
    # ],
    fieldsToCopy=[
        "width_min_",
        "width_mean",
        "width_max_"
    ],
    max_distance=None,
    neighbors=1,
    discardNonMatching=False,
    prefix="",
):
    """
    Join attributes by nearest
    Reference:
        https://docs.qgis.org/3.28/en/docs/user_manual/processing_algs/qgis/vectoroverlay.html#join-attributes-by-nearest

    Parameters
    ----------
    input1 : str or Path object
        Path to the input shapefile (reach lines)
    input2 : str or Path object
        Path to the input shapefile (shapefile with attributes to join; http://gaia.geosci.unc.edu/rivers/)
    output : str or Path object
        Path to the output shapefile (reach lines with attributes joined)
    fieldsToCopy : list
        List of fields to copy from input2 to input1
    max_distance : int or float
        Maximum distance to search for nearest features
    neighbors : int
        Number of nearest features to search for
    discardNonMatching : bool
        Discard features that do not have a match
    prefix : str
        Prefix to add to the field names of input2

    Returns
    -------
    None
    """
    processing.run(
        "native:joinbynearest",
        {
            "INPUT": str(input1),
            "INPUT_2": str(input2),
            "FIELDS_TO_COPY": fieldsToCopy,
            "DISCARD_NONMATCHING": discardNonMatching,
            "PREFIX": prefix,
            "NEIGHBORS": neighbors,
            "MAX_DISTANCE": max_distance,
            "OUTPUT": str(output),
        },
    )


def aggregateAttributes(input, output, aggregates):
    processing.run(
        "native:aggregate",
        {
            "INPUT": str(input),
            "GROUP_BY": '"uniqueID"',
            "AGGREGATES": aggregates,
            "OUTPUT": str(output),
        },
    )


def fieldCalculator(
    input,
    field_name="uniqueID",
    field_type=1,
    field_length=0,
    field_precision=0,
    formula="@id",
):
    output = Path(input).parents[0] / "temp.shp"

    processing.run(
        "native:fieldcalculator",
        {
            "INPUT": str(input),
            "FIELD_NAME": field_name,
            "FIELD_TYPE": field_type,
            "FIELD_LENGTH": field_length,
            "FIELD_PRECISION": field_precision,
            "FORMULA": formula,
            "OUTPUT": str(output),
        },
    )

    # save features back to the original file
    processing.run(
        "native:savefeatures",
        {
            "INPUT": str(output),
            "OUTPUT": str(input),
            "LAYER_NAME": "",
            "DATASOURCE_OPTIONS": "",
            "LAYER_OPTIONS": "",
        },
    )


def filterReachLines(input, overlay, output):
    """
    Filter reach lines to remove lines that overlap with reservoirs

    Parameters
    ----------
    input : str or Path object
        Path to the input shapefile (reach lines)
    overlay : str or Path object
        Path to the overlay shapefile (reservoirs)
    output : str or Path object
        Path to the output shapefile (filtered reach lines)

    Returns
    -------
    None
    """

    processing.run(
        "native:difference",
        {
            "INPUT": str(input),
            "OVERLAY": str(overlay),
            "OUTPUT": str(output),
            "GRID_SIZE": None,
        },
    )


def createBufferedReaches(input, output, bufferDistance=120):
    """
    Create buffered reaches from a line
    Reference:
        https://docs.qgis.org/3.28/en/docs/user_manual/processing_algs/qgis/vectorgeometry.html#buffer
        https://gis.stackexchange.com/questions/380361/creating-perpendicular-lines-on-line-using-qgis

    Parameters
    ----------
    input : str or Path object
        Path to the input shapefile (reach lines of the river)
    output : str or Path object
        Path to the output shapefile (buffered reaches)
    distance : int or float
        Distance to buffer the reach lines (in meters). Recommendd: the average width of the river.
    """
    processing.run(
        "native:buffer",
        {
            "INPUT": str(input),
            "DISTANCE": QgsProperty.fromExpression(f'"WIDTH" + (2*{bufferDistance})'),
            "SEGMENTS": 5,
            "END_CAP_STYLE": 0,  # Flat ends ()
            "JOIN_STYLE": 0,
            "MITER_LIMIT": 2,
            "DISSOLVE": False,
            "OUTPUT": str(output),
        },
    )


# joinAttributesByNearest(temporaryFile, narivs, reachLines)
def extractReachDimensions(input, dimensionFile, aggregates):
    joinAttributesByNearest(input, dimensionFile, temporaryFile, max_distance=1000)
    aggregateAttributes(temporaryFile, input, aggregates)

In [4]:
aggregates_narvis = [
    {
        "aggregate": "first_value",
        "delimiter": ",",
        "input": '"GNIS_ID"',
        "length": 10,
        "name": "GNIS_ID",
        "precision": 0,
        "sub_type": 0,
        "type": 10,
        "type_name": "text",
    },
    {
        "aggregate": "first_value",
        "delimiter": ",",
        "input": '"GNIS_Name"',
        "length": 255,
        "name": "GNIS_Name",
        "precision": 0,
        "sub_type": 0,
        "type": 10,
        "type_name": "text",
    },
    {
        "aggregate": "first_value",
        "delimiter": ",",
        "input": '"Basin"',
        "length": 255,
        "name": "Basin",
        "precision": 0,
        "sub_type": 0,
        "type": 10,
        "type_name": "text",
    },
    {
        "aggregate": "first_value",
        "delimiter": ",",
        "input": '"uniqueID"',
        "length": 10,
        "name": "uniqueID",
        "precision": 0,
        "sub_type": 0,
        "type": 4,
        "type_name": "int8",
    },
    {
        "aggregate": "mean",
        "delimiter": ",",
        "input": '"DISCHARGE"',
        "length": 23,
        "name": "DISCHARGE",
        "precision": 15,
        "sub_type": 0,
        "type": 6,
        "type_name": "double precision",
    },
    {
        "aggregate": "mean",
        "delimiter": ",",
        "input": '"WIDTH"',
        "length": 23,
        "name": "WIDTH",
        "precision": 15,
        "sub_type": 0,
        "type": 6,
        "type_name": "double precision",
    },
    {
        "aggregate": "mean",
        "delimiter": ",",
        "input": '"WIDTH5"',
        "length": 23,
        "name": "WIDTH5",
        "precision": 15,
        "sub_type": 0,
        "type": 6,
        "type_name": "double precision",
    },
    {
        "aggregate": "mean",
        "delimiter": ",",
        "input": '"WIDTH95"',
        "length": 23,
        "name": "WIDTH95",
        "precision": 15,
        "sub_type": 0,
        "type": 6,
        "type_name": "double precision",
    },
    {
        "aggregate": "mean",
        "delimiter": ",",
        "input": '"DEPTH"',
        "length": 23,
        "name": "DEPTH",
        "precision": 15,
        "sub_type": 0,
        "type": 6,
        "type_name": "double precision",
    },
    {
        "aggregate": "mean",
        "delimiter": ",",
        "input": '"DEPTH5"',
        "length": 23,
        "name": "DEPTH5",
        "precision": 15,
        "sub_type": 0,
        "type": 6,
        "type_name": "double precision",
    },
    {
        "aggregate": "mean",
        "delimiter": ",",
        "input": '"DEPTH95"',
        "length": 23,
        "name": "DEPTH95",
        "precision": 15,
        "sub_type": 0,
        "type": 6,
        "type_name": "double precision",
    },
    {
        "aggregate": "mean",
        "delimiter": ",",
        "input": '"n"',
        "length": 0,
        "name": "n",
        "precision": 0,
        "sub_type": 0,
        "type": 2,
        "type_name": "integer",
    },
    {
        "aggregate": "mean",
        "delimiter": ",",
        "input": '"distance"',
        "length": 0,
        "name": "distance",
        "precision": 0,
        "sub_type": 0,
        "type": 6,
        "type_name": "double precision",
    },
    {
        "aggregate": "mean",
        "delimiter": ",",
        "input": '"feature_x"',
        "length": 0,
        "name": "feature_x",
        "precision": 0,
        "sub_type": 0,
        "type": 6,
        "type_name": "double precision",
    },
    {
        "aggregate": "mean",
        "delimiter": ",",
        "input": '"feature_y"',
        "length": 0,
        "name": "feature_y",
        "precision": 0,
        "sub_type": 0,
        "type": 6,
        "type_name": "double precision",
    },
    {
        "aggregate": "mean",
        "delimiter": ",",
        "input": '"nearest_x"',
        "length": 0,
        "name": "nearest_x",
        "precision": 0,
        "sub_type": 0,
        "type": 6,
        "type_name": "double precision",
    },
    {
        "aggregate": "mean",
        "delimiter": ",",
        "input": '"nearest_y"',
        "length": 0,
        "name": "nearest_y",
        "precision": 0,
        "sub_type": 0,
        "type": 6,
        "type_name": "double precision",
    },
]

aggregates_grwl = [
    {
        "aggregate": "first_value",
        "delimiter": ",",
        "input": '"GNIS_ID"',
        "length": 10,
        "name": "GNIS_ID",
        "precision": 0,
        "sub_type": 0,
        "type": 10,
        "type_name": "text",
    },
    {
        "aggregate": "first_value",
        "delimiter": ",",
        "input": '"GNIS_Name"',
        "length": 255,
        "name": "GNIS_Name",
        "precision": 0,
        "sub_type": 0,
        "type": 10,
        "type_name": "text",
    },
    {
        "aggregate": "first_value",
        "delimiter": ",",
        "input": '"Basin"',
        "length": 255,
        "name": "Basin",
        "precision": 0,
        "sub_type": 0,
        "type": 10,
        "type_name": "text",
    },
    {
        "aggregate": "first_value",
        "delimiter": ",",
        "input": '"uniqueID"',
        "length": 10,
        "name": "uniqueID",
        "precision": 0,
        "sub_type": 0,
        "type": 4,
        "type_name": "int8",
    },
    {
        "aggregate": "mean",
        "delimiter": ",",
        "input": '"width_min_"',
        "length": 23,
        "name": "WidthMin",
        "precision": 15,
        "sub_type": 0,
        "type": 6,
        "type_name": "double precision",
    },
    {
        "aggregate": "mean",
        "delimiter": ",",
        "input": '"width_mean"',
        "length": 23,
        "name": "WidthMean",
        "precision": 15,
        "sub_type": 0,
        "type": 6,
        "type_name": "double precision",
    },
    {
        "aggregate": "mean",
        "delimiter": ",",
        "input": '"width_max_"',
        "length": 23,
        "name": "WidthMax",
        "precision": 15,
        "sub_type": 0,
        "type": 6,
        "type_name": "double precision",
    },
]

In [5]:
# test the functions
interpolatePoints(input_flowline, interpolatedPoints)
print("Interpolated points created")
createTransects(interpolatedPoints, transects)
print("Transects created")
createReachLines(input_flowline, transects, reachLines)
print("Reach lines created")
fieldCalculator(reachLines)
print("Unique ID field created")
extractReachDimensions(reachLines, grwl, aggregates_grwl)
print("Reach dimensions extracted")
filterReachLines(reachLines, reservoirs_shapefile, filteredReachLines)
print("Reach lines filtered")

Interpolated points created
Transects created
Reach lines created
Unique ID field created


Warning 1: Value -13393695.0024622791 of field feature_x of feature 12 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value -13393695.0024622791 of field nearest_x of feature 12 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value -12956795.0568334945 of field feature_x of feature 29 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value -12956795.0568334945 of field nearest_x of feature 29 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value -13148570.2271600813 of field feature_x of feature 43 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value -13149190.7226975691 of field nearest_x of feature 43 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value -13162895.5434855651 of field

Reach dimensions extracted
Reach lines filtered
